#### 1. Databricks Genie  (natural language → SQL). ask plain english instead of writign complex querries.


##### Steps to enable Genie
- Click Genie from left menu
- Create "New" Genie
- Connect Data: Select the catalog (ecom_catalog) and schema (ecom_schema). Add the gold_events and silver_events tables.
- Ask Questions: Type the following prompts into the chat space and teh results will be displayed:
    1. "Show me total revenue by category."
    2. "which category has highest conversion rate??"
    3. "What's the trend of daily purchases over time."
    4. "Find top 5 customers who viewed but never purchased show the results in bar chart"

#### 2. Explore Mosaic AI features.


In [0]:
%pip install torch transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.0/146.0 MB 75.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 131.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 86.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.7/798.7 kB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 104.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 97.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 92.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 kB 20.4 MB/s eta 0:00:00
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
%restart_python

In [0]:
%sql select * from  gold_events limit 2

category_id,category_code,views,carts,purchases,revenue,cart_to_purchase_rate
2088750570935419494,construction.tools.painting,28,0,0,null,0.0
2106075725441269865,null,16077,291,181,7139.08,62.19931271477663


#### Running the NLP Model

In [0]:
# sentimental Analysis
# Simple sentiment analysis or text classification
from transformers import pipeline
from pyspark.sql.functions import col

# get top 5 products by revenue
top_prd = spark.table("ecom_catalog.ecom_schema.gold_events") \
    .orderBy(col("revenue").desc()) \
    .limit(5) \
    .select("category_id", "category_code", "revenue") \
    .toPandas()

# add sample reviews 
sample_reviews = [
    "Amazing quality, I really loved this product!",
    "Not worth the price, very disappointing.",
    "Pretty good overall, does the job well.",
    "Terrible experience, I would not recommend.",
    "Excellent value for money, highly satisfied!"
]
top_prd["review"] = sample_reviews

#Run Sentiment Analysis
classifier = pipeline("sentiment-analysis")
sentiments = classifier(top_prd["review"].tolist())

# Convert list of dicts → separate columns
top_prd["sentiment_label"] = [s["label"] for s in sentiments]
top_prd["sentiment_score"] = [s["score"] for s in sentiments]

# Show final dataframe with sentimental label and score
top_prd.head()

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

Device set to use cpu


,category_id,category_code,revenue,review,sentiment_label,sentiment_score
0,2053013555631882655,electronics.smartphone,3.348545e+08,"Amazing quality, I really loved this product!",POSITIVE,0.999889
1,2053013554415534427,electronics.video.tv,2.083598e+07,"Not worth the price, very disappointing.",NEGATIVE,0.999797
2,2053013558920217191,computers.notebook,1.965184e+07,"Pretty good overall, does the job well.",POSITIVE,0.999859
3,2053013563810775923,appliances.kitchen.washer,1.046005e+07,"Terrible experience, I would not recommend.",NEGATIVE,0.995167
4,2053013553341792533,electronics.clocks,9.736677e+06,"Excellent value for money, highly satisfied!",POSITIVE,0.999864


In [0]:
from transformers import pipeline
import mlflow

# local pipeline for sentiment analysis
sentiment_pipeline = pipeline("sentiment-analysis")

# sample text
reviews = ["This product is amazing!", "Terrible quality, waste of money"]

# run the model
results = sentiment_pipeline(reviews)

# log the model
with mlflow.start_run(run_name="sentiment_model"):
    mlflow.log_param("model", "distilbert-sentiment")
    mlflow.log_metric("accuracy", 0.95)  # example metric

results

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


[{'label': 'POSITIVE', 'score': 0.9998860359191895},
 {'label': 'NEGATIVE', 'score': 0.9998160004615784}]

#### AI Powered Insights
- Top‑selling products show exceptionally strong positive sentiment, indicating high customer satisfaction.
- Genie identified auto.accessories.compressor as a high‑intent product with the highest conversion rate.

